In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings

from sklearn.decomposition import PCA
from lightgbm import LGBMClassifier
from sklearn.manifold import TSNE
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier,SGDClassifier,PassiveAggressiveClassifier,LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,StackingClassifier,ExtraTreesClassifier
from sklearn.feature_selection import RFECV,chi2,SelectKBest, SelectPercentile
from sklearn.gaussian_process import GaussianProcessClassifier
from xgboost.sklearn import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [ ]:
root_dir= "data/"
nsample = 3000
features = list()
with open('features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]
print('No of Features: {}'.format(len(features)))

In [ ]:
for i in range(1,100):
    X_train = pd.read_csv(root_dir+"X_train.csv",header=None)
    X_train.columns=features
    # X_train['subject'] = pd.read_csv('subject_train.csv',header=None)

    y_train = pd.read_csv(root_dir+"y_train.csv",names=['Activity'],header=None, squeeze=True)

    X_test = pd.read_csv(root_dir+"X_test.csv",header=None)
    X_test.columns=features

    # standard_scaler = StandardScaler()
    # standard_scaler.fit(X_train)
    # X = standard_scaler.transform(X_train)
    y = y_train


    X_train,X_val, y_train,y_val= train_test_split(X_train, y_train, test_size=0.2, random_state=i)

    y_train = np.array(y_train).flatten()
    X_train = np.array(X_train)

    y_val = np.array(y_val).flatten()
    X_val = np.array(X_val)

    X_test = np.array(X_test)

    # X_test = standard_scaler.transform(X_test)


    model = LGBMClassifier(max_depth=15)
    model.fit(X_train, y_train)
    yHatTrain = model.predict(X_train)
    yHatVal = model.predict(X_val)
    print('state: {}, Train: {:.6f}, Val: {:.6f}'.format(i, len((np.where(yHatTrain == y_train))[0])*1.0/X_train.shape[0],
                        len((np.where(yHatVal == y_val))[0])*1.0/X_val.shape[0]))
    yHatTest = model.predict(X_test)
    np.savetxt(root_dir+'result_state'+ str(i) + '.txt', yHatTest)